In [6]:
import pandas as pd

In [7]:
# Load all CSV files into dataframes

df_utilities = pd.read_csv('./datafiles/utilities.csv', low_memory=False)
df_suppliers = pd.read_csv('./datafiles/suppliers.csv', low_memory=False)
df_plans = pd.read_csv('./datafiles/plans.csv', low_memory=False)
df_rates = pd.read_csv('./datafiles/rates.csv', low_memory=False)
df_amounts = pd.read_csv('./datafiles/amounts.csv', low_memory=False)
df_monthly_charges = pd.read_csv('./datafiles/monthly_charges.csv', low_memory=False)
df_usage_credits = pd.read_csv('./datafiles/usage_credits.csv', low_memory=False)
df_early_term_fees = pd.read_csv('./datafiles/early_term_fees.csv', low_memory=False)
df_benefits = pd.read_csv('./datafiles/benefits.csv', low_memory=False)

In [8]:
df_utilities.rename(columns={'id': 'utility_id', 'name': 'utility_name'}, inplace=True)
df_suppliers.rename(columns={'id': 'supplier_id', 'name': 'supplier_name'}, inplace=True)
df_plans.rename(columns={'id': 'plan_id'}, inplace=True)
df_rates.rename(columns={'id': 'rate_id'}, inplace=True)
df_amounts.rename(columns={'id': 'amount_id'}, inplace=True)
df_monthly_charges.rename(columns={'id': 'monthly_charge_id'}, inplace=True)
df_usage_credits.rename(columns={'id': 'usage_credit_id'}, inplace=True)
df_early_term_fees.rename(columns={'id': 'early_term_fee_id'}, inplace=True)
df_benefits.rename(columns={'id': 'benefit_id'}, inplace=True)

In [9]:
df_plans_missing = df_plans [df_plans ['created_at'].isnull()]
df_plans_missing

,plan_id,created_at,state,rate_unit,contact_number,is_variable,rate_type,is_green,description,term,...,green_details,green_percentage,start_date,service_type,term_end_date,plan_end_date,sq_ft_2600,sq_ft_800,plan_type,supplier_id
22278,22279,NaN,pa,kWh,1-888-313-9086,False,fixed,False,NaN,6.0,...,25% renewable,25.0,NaN,NaN,NaN,NaN,NaN,NaN,Regular Residential Service,50
26694,26695,NaN,pa,kWh,1-888-313-9086,False,fixed,False,NaN,6.0,...,25% renewable,25.0,NaN,Residential,NaN,NaN,NaN,NaN,NaN,50
29202,29203,NaN,pa,kWh,NaN,True,variable,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular Residential Service,85
29203,29204,NaN,pa,kWh,NaN,NaN,fixed,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular Residential Service,85
29204,29205,NaN,pa,kWh,NaN,NaN,fixed,False,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Regular Residential Service,85
29205,29206,NaN,pa,kWh,NaN,NaN,fixed,True,NaN,12.0,...,wind,100.0,NaN,NaN,NaN,NaN,NaN,NaN,Regular Residential Service,85


In [10]:
df_rates_missing = df_rates [df_rates ['createdAt'].isnull()]
df_rates_missing

,rate_id,rate_type,min_value,max_value,utility_id,plan_id,createdAt
16484,16485,default rate,0,NaN,8.0,NaN,NaN
20543,20544,default rate,0,NaN,8.0,NaN,NaN
21102,21103,default rate,0,NaN,8.0,NaN,NaN
21599,21600,default rate,0,NaN,8.0,NaN,NaN
22140,22141,default rate,0,NaN,8.0,NaN,NaN
22555,22556,default rate,0,NaN,8.0,NaN,NaN
23080,23081,default rate,0,NaN,8.0,NaN,NaN
23557,23558,default rate,0,NaN,8.0,NaN,NaN
24039,24040,default rate,0,NaN,8.0,NaN,NaN
24599,24600,default rate,0,NaN,8.0,NaN,NaN


In [11]:
# drop plans with missing created_at column
df_plans = df_plans.dropna(subset=['created_at'])
df_rates = df_rates.dropna(subset=['createdAt'])

In [12]:
# Add 'rate_source' column to df_rates
df_rates['rate_source'] = df_rates.apply(
    lambda x: 'utility_rate' if pd.notnull(x['utility_id']) else 'plan_rate', axis=1
)

In [13]:
# Split df_rates into utility rates and plan rates
df_utility_rates = df_rates[df_rates['rate_source'] == 'utility_rate']
df_plan_rates = df_rates[df_rates['rate_source'] == 'plan_rate']

In [14]:
# Merge df_plan_rates with df_plans
df_plan_rates = df_plan_rates.merge(df_plans, how='left', on='plan_id', suffixes=('', '_plan'))

In [15]:
# merge df_plan_rates with df_suppliers on 'supplier_id'
df_plan_rates = df_plan_rates.merge(df_suppliers[['supplier_id', 'supplier_name', 'utility_id']], how='left', on='supplier_id')

In [16]:
df_plan_rates.rename(columns={
    'utility_id_y': 'utility_id',
}, inplace=True)

In [17]:
# add utility name and utility names
df_plan_rates = df_plan_rates.merge(df_utilities[['utility_id', 'utility_name']], how='left', on='utility_id')


In [18]:
missing_plan_dates_df= df_plan_rates [df_plan_rates ['createdAt'].isnull()]

# Display the filtered DataFrame
print("Plans with missing 'plan_rate_created_at':")
missing_plan_dates_df


Plans with missing 'plan_rate_created_at':


,rate_id,rate_type,min_value,max_value,utility_id_x,plan_id,createdAt,rate_source,created_at,state,...,service_type,term_end_date,plan_end_date,sq_ft_2600,sq_ft_800,plan_type,supplier_id,supplier_name,utility_id,utility_name


In [19]:
# drop rows that have a missing

In [20]:
# get amounts associated with plan rates
df_amounts_plan = df_amounts[df_amounts['rate_id'].isin(df_plan_rates['rate_id'])]

In [21]:
# Merge amounts with plan rates
df_plan_rates = df_plan_rates.merge(df_amounts_plan, how='left', on='rate_id')

In [22]:
df_plan_rates.rename(columns={'value': 'plan_rate_amount'}, inplace=True)

In [23]:
df_plan_rates

,rate_id,rate_type,min_value,max_value,utility_id_x,plan_id,createdAt,rate_source,created_at,state,...,supplier_id,supplier_name,utility_id,utility_name,amount_id,from_value,to_value,plan_rate_amount,usage_credit_id,monthly_charge_id
0,2,NaN,0,NaN,NaN,1.0,1999-07-30,plan_rate,1999-07-30,pa,...,1,FirstEnergy,1,Allegheny Power,2.0,NaN,NaN,4.40,NaN,NaN
1,3,NaN,0,NaN,NaN,2.0,1999-07-30,plan_rate,1999-07-30,pa,...,2,Green Mountain Energy,1,Allegheny Power,4.0,NaN,NaN,4.04,NaN,NaN
2,4,NaN,0,1200,NaN,3.0,1999-07-30,plan_rate,1999-07-30,pa,...,2,Green Mountain Energy,1,Allegheny Power,6.0,NaN,NaN,5.06,NaN,NaN
3,5,NaN,1201,NaN,NaN,3.0,1999-07-30,plan_rate,1999-07-30,pa,...,2,Green Mountain Energy,1,Allegheny Power,7.0,NaN,NaN,4.23,NaN,NaN
4,6,NaN,0,1200,NaN,4.0,1999-07-30,plan_rate,1999-07-30,pa,...,2,Green Mountain Energy,1,Allegheny Power,9.0,NaN,NaN,5.70,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656488,659375,NaN,0,NaN,NaN,656737.0,2021-07-21,plan_rate,2021-07-21,pa,...,58,Ambit Energy,3,PECO Energy,1773845.0,NaN,NaN,6.95,NaN,NaN
656489,659376,NaN,0,NaN,NaN,656738.0,2021-07-21,plan_rate,2021-07-21,pa,...,58,Ambit Energy,3,PECO Energy,1773848.0,NaN,NaN,6.95,NaN,NaN
656490,659377,NaN,0,NaN,NaN,656739.0,2021-07-21,plan_rate,2021-07-21,pa,...,94,Energy Rewards,4,Penelec,1773851.0,NaN,NaN,8.69,NaN,NaN
656491,659378,NaN,0,NaN,NaN,656740.0,2021-07-21,plan_rate,2021-07-21,pa,...,94,Energy Rewards,4,Penelec,1773854.0,NaN,NaN,8.59,NaN,NaN


In [24]:
df_plan_rates.columns

Index(['rate_id', 'rate_type', 'min_value', 'max_value', 'utility_id_x',
       'plan_id', 'createdAt', 'rate_source', 'created_at', 'state',
       'rate_unit', 'contact_number', 'is_variable', 'rate_type_plan',
       'is_green', 'description', 'term', 'website_url', 'green_details',
       'green_percentage', 'start_date', 'service_type', 'term_end_date',
       'plan_end_date', 'sq_ft_2600', 'sq_ft_800', 'plan_type', 'supplier_id',
       'supplier_name', 'utility_id', 'utility_name', 'amount_id',
       'from_value', 'to_value', 'plan_rate_amount', 'usage_credit_id',
       'monthly_charge_id'],
      dtype='object')

In [25]:
# Specify the columns to drop as a list
columns_to_drop = ['rate_id', 'rate_type', 'utility_id_x', 'rate_source',
        'amount_id', 'usage_credit_id','created_at','monthly_charge_id']

In [26]:
# Drop the columns and update the DataFrame in place
df_plan_rates.drop(columns=columns_to_drop, inplace=True)

In [27]:
df_plan_rates

,min_value,max_value,plan_id,createdAt,state,rate_unit,contact_number,is_variable,rate_type_plan,is_green,...,sq_ft_2600,sq_ft_800,plan_type,supplier_id,supplier_name,utility_id,utility_name,from_value,to_value,plan_rate_amount
0,0,NaN,1.0,1999-07-30,pa,kWh,888-254-9227,True,variable,False,...,NaN,NaN,Regular Residential Service,1,FirstEnergy,1,Allegheny Power,NaN,NaN,4.40
1,0,NaN,2.0,1999-07-30,pa,kWh,888-246-6730,True,variable,False,...,NaN,NaN,Regular Residential Service,2,Green Mountain Energy,1,Allegheny Power,NaN,NaN,4.04
2,0,1200,3.0,1999-07-30,pa,kWh,888-246-6730,True,variable,False,...,NaN,NaN,Regular Residential Service,2,Green Mountain Energy,1,Allegheny Power,NaN,NaN,5.06
3,1201,NaN,3.0,1999-07-30,pa,kWh,888-246-6730,True,variable,False,...,NaN,NaN,Regular Residential Service,2,Green Mountain Energy,1,Allegheny Power,NaN,NaN,4.23
4,0,1200,4.0,1999-07-30,pa,kWh,888-246-6730,True,variable,True,...,NaN,NaN,Regular Residential Service,2,Green Mountain Energy,1,Allegheny Power,NaN,NaN,5.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656488,0,NaN,656737.0,2021-07-21,pa,kWh,NaN,NaN,fixed,False,...,6.95,6.95,NaN,58,Ambit Energy,3,PECO Energy,NaN,NaN,6.95
656489,0,NaN,656738.0,2021-07-21,pa,kWh,NaN,NaN,fixed,False,...,6.95,6.95,NaN,58,Ambit Energy,3,PECO Energy,NaN,NaN,6.95
656490,0,NaN,656739.0,2021-07-21,pa,kWh,NaN,NaN,fixed,False,...,8.69,8.69,NaN,94,Energy Rewards,4,Penelec,NaN,NaN,8.69
656491,0,NaN,656740.0,2021-07-21,pa,kWh,NaN,NaN,fixed,False,...,8.59,8.59,NaN,94,Energy Rewards,4,Penelec,NaN,NaN,8.59


In [28]:
# Rename columns to include 'plan_rate_' prefix
df_plan_rates.rename(columns={
    'min_value': 'plan_rate_min_value',
    'max_value': 'plan_rate_max_value',
    'createdAt': 'created_at',
    'from_value': 'plan_rate_from_value',
    'to_value': 'plan_rate_to_value'
}, inplace=True)

In [29]:
df_plan_rates.columns

Index(['plan_rate_min_value', 'plan_rate_max_value', 'plan_id', 'created_at',
       'state', 'rate_unit', 'contact_number', 'is_variable', 'rate_type_plan',
       'is_green', 'description', 'term', 'website_url', 'green_details',
       'green_percentage', 'start_date', 'service_type', 'term_end_date',
       'plan_end_date', 'sq_ft_2600', 'sq_ft_800', 'plan_type', 'supplier_id',
       'supplier_name', 'utility_id', 'utility_name', 'plan_rate_from_value',
       'plan_rate_to_value', 'plan_rate_amount'],
      dtype='object')

In [30]:
# Define the columns to keep
plan_columns = [
    'utility_id',
    'utility_name',
    'supplier_id',
    'supplier_name',
    'plan_id',
    'created_at',
    'state',
    'rate_unit',
    'contact_number',
    'is_variable',
    'rate_type_plan',
    'is_green',
    'description',
    'term',
    'website_url',
    'green_details',
    'green_percentage',
    'start_date',
    'service_type',
    'term_end_date',
    'plan_end_date',
    'sq_ft_2600',
    'sq_ft_800',
    'plan_type',
    'plan_rate_min_value',
    'plan_rate_max_value',
    'plan_rate_from_value',
    'plan_rate_to_value',
    'plan_rate_amount',
]

In [31]:
# Select the columns
df_utility_plan_rates_amounts = df_plan_rates[plan_columns]

In [32]:
df_utility_plan_rates_amounts

,utility_id,utility_name,supplier_id,supplier_name,plan_id,created_at,state,rate_unit,contact_number,is_variable,...,term_end_date,plan_end_date,sq_ft_2600,sq_ft_800,plan_type,plan_rate_min_value,plan_rate_max_value,plan_rate_from_value,plan_rate_to_value,plan_rate_amount
0,1,Allegheny Power,1,FirstEnergy,1.0,1999-07-30,pa,kWh,888-254-9227,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,0,NaN,NaN,NaN,4.40
1,1,Allegheny Power,2,Green Mountain Energy,2.0,1999-07-30,pa,kWh,888-246-6730,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,0,NaN,NaN,NaN,4.04
2,1,Allegheny Power,2,Green Mountain Energy,3.0,1999-07-30,pa,kWh,888-246-6730,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,0,1200,NaN,NaN,5.06
3,1,Allegheny Power,2,Green Mountain Energy,3.0,1999-07-30,pa,kWh,888-246-6730,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,1201,NaN,NaN,NaN,4.23
4,1,Allegheny Power,2,Green Mountain Energy,4.0,1999-07-30,pa,kWh,888-246-6730,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,0,1200,NaN,NaN,5.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656488,3,PECO Energy,58,Ambit Energy,656737.0,2021-07-21,pa,kWh,NaN,NaN,...,NaN,NaN,6.95,6.95,NaN,0,NaN,NaN,NaN,6.95
656489,3,PECO Energy,58,Ambit Energy,656738.0,2021-07-21,pa,kWh,NaN,NaN,...,NaN,NaN,6.95,6.95,NaN,0,NaN,NaN,NaN,6.95
656490,4,Penelec,94,Energy Rewards,656739.0,2021-07-21,pa,kWh,NaN,NaN,...,NaN,NaN,8.69,8.69,NaN,0,NaN,NaN,NaN,8.69
656491,4,Penelec,94,Energy Rewards,656740.0,2021-07-21,pa,kWh,NaN,NaN,...,NaN,NaN,8.59,8.59,NaN,0,NaN,NaN,NaN,8.59


In [33]:
# Filter amounts associated with utility rates
df_amounts_utility = df_amounts[df_amounts['rate_id'].isin(df_utility_rates['rate_id'])]

# Merge amounts with utility rates
df_utility_rates = df_utility_rates.merge(df_amounts_utility, how='left', on='rate_id')

In [34]:
df_utility_rates

,rate_id,rate_type,min_value,max_value,utility_id,plan_id,createdAt,rate_source,amount_id,from_value,to_value,value,usage_credit_id,monthly_charge_id
0,1,default rate,0,NaN,1.0,NaN,1999-07-30,utility_rate,1,NaN,NaN,3.221,NaN,NaN
1,10,default rate,0,NaN,2.0,NaN,1999-07-30,utility_rate,16,NaN,NaN,4.522,NaN,NaN
2,25,default residential heat pump rate,0,500,3.0,NaN,1999-07-30,utility_rate,48,NaN,NaN,5.770,NaN,NaN
3,26,default residential heat pump rate,501,NaN,3.0,NaN,1999-07-30,utility_rate,49,NaN,NaN,6.470,NaN,NaN
4,47,default rate,0,NaN,4.0,NaN,1999-07-30,utility_rate,92,NaN,NaN,4.528,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2987,659498,default rate,0,NaN,3.0,NaN,2020-09-02,utility_rate,1773978,NaN,NaN,6.369,NaN,NaN
2988,659499,default rate,0,NaN,3.0,NaN,2020-09-02,utility_rate,1773979,NaN,NaN,6.405,NaN,NaN
2989,659500,default rate,0,NaN,3.0,NaN,2020-09-03,utility_rate,1773980,NaN,NaN,6.369,NaN,NaN
2990,659501,default rate,0,-,3.0,NaN,2023-09-14,utility_rate,1773981,NaN,NaN,8.917,NaN,NaN


In [35]:
df_utility_rates.columns

Index(['rate_id', 'rate_type', 'min_value', 'max_value', 'utility_id',
       'plan_id', 'createdAt', 'rate_source', 'amount_id', 'from_value',
       'to_value', 'value', 'usage_credit_id', 'monthly_charge_id'],
      dtype='object')

In [36]:
utility_rates_columns = [
    "utility_id","rate_type","createdAt","from_value","to_value","value"
]

# Select the columns
df_utility_rates_amounts = df_utility_rates[utility_rates_columns].copy()

In [37]:
df_utility_rates_amounts

,utility_id,rate_type,createdAt,from_value,to_value,value
0,1.0,default rate,1999-07-30,NaN,NaN,3.221
1,2.0,default rate,1999-07-30,NaN,NaN,4.522
2,3.0,default residential heat pump rate,1999-07-30,NaN,NaN,5.770
3,3.0,default residential heat pump rate,1999-07-30,NaN,NaN,6.470
4,4.0,default rate,1999-07-30,NaN,NaN,4.528
...,...,...,...,...,...,...
2987,3.0,default rate,2020-09-02,NaN,NaN,6.369
2988,3.0,default rate,2020-09-02,NaN,NaN,6.405
2989,3.0,default rate,2020-09-03,NaN,NaN,6.369
2990,3.0,default rate,2023-09-14,NaN,NaN,8.917


In [38]:
# Rename columns to include 'plan_rate_' prefix
df_utility_rates_amounts.rename(columns={
    'createdAt': 'created_at',
    'from_value': 'utility_rate_from_value',
    'to_value': 'utility_rate_to_value',
    'value': 'utility_rate_value',
    'rate_type': 'utility_rate_type',
}, inplace=True)

In [39]:
df_utility_rates_amounts

,utility_id,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_value
0,1.0,default rate,1999-07-30,NaN,NaN,3.221
1,2.0,default rate,1999-07-30,NaN,NaN,4.522
2,3.0,default residential heat pump rate,1999-07-30,NaN,NaN,5.770
3,3.0,default residential heat pump rate,1999-07-30,NaN,NaN,6.470
4,4.0,default rate,1999-07-30,NaN,NaN,4.528
...,...,...,...,...,...,...
2987,3.0,default rate,2020-09-02,NaN,NaN,6.369
2988,3.0,default rate,2020-09-02,NaN,NaN,6.405
2989,3.0,default rate,2020-09-03,NaN,NaN,6.369
2990,3.0,default rate,2023-09-14,NaN,NaN,8.917


In [40]:
# add names to utility _rates_amounts
df_utility_rates_amounts = pd.merge(df_utilities,df_utility_rates_amounts,on="utility_id")

In [41]:
df_utility_rates_amounts

,utility_id,utility_name,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_value
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221
2,1,Allegheny Power,default rate,1999-09-30,NaN,NaN,3.221
3,1,Allegheny Power,default rate,2000-03-31,NaN,NaN,3.243
4,1,Allegheny Power,default rate,2000-07-31,NaN,NaN,3.243
...,...,...,...,...,...,...,...
2987,10,West Penn Power,default rate,2019-09-01,NaN,NaN,5.338
2988,10,West Penn Power,default rate,2019-12-01,NaN,NaN,5.760
2989,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059
2990,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929


In [42]:
# Convert 'plan_rate_created_at' to datetime
df_utility_plan_rates_amounts['created_at'] = pd.to_datetime(df_utility_plan_rates_amounts['created_at'])

# Convert 'utility_created_at' to datetime
df_utility_rates_amounts['created_at'] = pd.to_datetime(df_utility_rates_amounts['created_at'])

In [43]:
# Sort df_utility_rates_amounts by 'utility_id' and 'utility_created_at'
df_utility_rates_amounts_sorted = df_utility_rates_amounts.sort_values(['utility_id', 'created_at'])

# Sort df_utility_plan_rates_amounts by 'utility_id' and 'plan_rate_created_at'
df_utility_plan_rates_amounts_sorted = df_utility_plan_rates_amounts.sort_values(['utility_id', 'created_at'])

In [44]:
# Convert 'utility_id' to int64 in df_utility_rates_amounts
df_utility_rates_amounts_sorted['utility_id'] = df_utility_rates_amounts_sorted['utility_id'].astype('int64')

# Convert 'utility_id' to int64 in df_utility_plan_rates_amounts
df_utility_plan_rates_amounts_sorted['utility_id'] = df_utility_plan_rates_amounts_sorted['utility_id'].astype('int64')

In [45]:
print("Data types in df_utility_rates_amounts:")
print(df_utility_rates_amounts_sorted[['utility_id']].dtypes)

# Check data types of 'utility_id' in df_utility_plan_rates_amounts
print("\nData types in df_utility_plan_rates_amounts:")
print(df_utility_plan_rates_amounts_sorted[['utility_id']].dtypes)

Data types in df_utility_rates_amounts:
utility_id    int64
dtype: object

Data types in df_utility_plan_rates_amounts:
utility_id    int64
dtype: object


In [46]:
df_utility_rates_amounts_sorted.columns

Index(['utility_id', 'utility_name', 'utility_rate_type', 'created_at',
       'utility_rate_from_value', 'utility_rate_to_value',
       'utility_rate_value'],
      dtype='object')

In [47]:
df_utility_plan_rates_amounts_sorted.columns

Index(['utility_id', 'utility_name', 'supplier_id', 'supplier_name', 'plan_id',
       'created_at', 'state', 'rate_unit', 'contact_number', 'is_variable',
       'rate_type_plan', 'is_green', 'description', 'term', 'website_url',
       'green_details', 'green_percentage', 'start_date', 'service_type',
       'term_end_date', 'plan_end_date', 'sq_ft_2600', 'sq_ft_800',
       'plan_type', 'plan_rate_min_value', 'plan_rate_max_value',
       'plan_rate_from_value', 'plan_rate_to_value', 'plan_rate_amount'],
      dtype='object')

In [48]:
df_utility_rates_amounts_sorted

,utility_id,utility_name,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_value
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221
2,1,Allegheny Power,default rate,1999-09-30,NaN,NaN,3.221
6,1,Allegheny Power,default rate,2000-02-25,NaN,NaN,3.243
7,1,Allegheny Power,default rate,2000-02-25,NaN,NaN,3.243
...,...,...,...,...,...,...,...
2987,10,West Penn Power,default rate,2019-09-01,NaN,NaN,5.338
2988,10,West Penn Power,default rate,2019-12-01,NaN,NaN,5.760
2989,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059
2990,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929


In [49]:
df_utility_plan_rates_amounts_sorted

,utility_id,utility_name,supplier_id,supplier_name,plan_id,created_at,state,rate_unit,contact_number,is_variable,...,term_end_date,plan_end_date,sq_ft_2600,sq_ft_800,plan_type,plan_rate_min_value,plan_rate_max_value,plan_rate_from_value,plan_rate_to_value,plan_rate_amount
0,1,Allegheny Power,1,FirstEnergy,1.0,1999-07-30,pa,kWh,888-254-9227,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,0,NaN,NaN,NaN,4.40
1,1,Allegheny Power,2,Green Mountain Energy,2.0,1999-07-30,pa,kWh,888-246-6730,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,0,NaN,NaN,NaN,4.04
2,1,Allegheny Power,2,Green Mountain Energy,3.0,1999-07-30,pa,kWh,888-246-6730,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,0,1200,NaN,NaN,5.06
3,1,Allegheny Power,2,Green Mountain Energy,3.0,1999-07-30,pa,kWh,888-246-6730,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,1201,NaN,NaN,NaN,4.23
4,1,Allegheny Power,2,Green Mountain Energy,4.0,1999-07-30,pa,kWh,888-246-6730,True,...,NaN,NaN,NaN,NaN,Regular Residential Service,0,1200,NaN,NaN,5.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656478,10,West Penn Power,115,Tomorrow Energy,656727.0,2021-07-21,pa,kWh,NaN,NaN,...,NaN,NaN,7.67,7.67,existing,0,NaN,NaN,NaN,7.67
656479,10,West Penn Power,96,Town Square Energy,656728.0,2021-07-21,pa,kWh,NaN,NaN,...,NaN,NaN,6.04,6.04,NaN,0,NaN,NaN,NaN,6.04
656480,10,West Penn Power,96,Town Square Energy,656729.0,2021-07-21,pa,kWh,NaN,NaN,...,NaN,NaN,7.97,7.97,NaN,0,NaN,NaN,NaN,7.97
656481,10,West Penn Power,96,Town Square Energy,656730.0,2021-07-21,pa,kWh,NaN,NaN,...,NaN,NaN,7.97,7.97,NaN,0,NaN,NaN,NaN,7.97


In [50]:
# Check for missing 'utility_id' and date columns in df_utility_rates_amounts_sorted
missing_util_id_rates = df_utility_rates_amounts_sorted['utility_id'].isnull().sum()
missing_dates_rates = df_utility_rates_amounts_sorted['created_at'].isnull().sum()
print(f"Missing 'utility_id' in df_utility_rates_amounts_sorted: {missing_util_id_rates}")
print(f"Missing 'utility_created_at' in df_utility_rates_amounts_sorted: {missing_dates_rates}")

Missing 'utility_id' in df_utility_rates_amounts_sorted: 0
Missing 'utility_created_at' in df_utility_rates_amounts_sorted: 0


In [51]:
# Check for missing 'utility_id' and date columns in df_utility_plan_rates_amounts_sorted
missing_util_id_plan = df_utility_plan_rates_amounts_sorted['utility_id'].isnull().sum()
missing_dates_plan = df_utility_plan_rates_amounts_sorted['created_at'].isnull().sum()
print(f"\nMissing 'utility_id' in df_utility_plan_rates_amounts_sorted: {missing_util_id_plan}")
print(f"Missing 'plan_rate_created_at' in df_utility_plan_rates_amounts_sorted: {missing_dates_plan}")


Missing 'utility_id' in df_utility_plan_rates_amounts_sorted: 0
Missing 'plan_rate_created_at' in df_utility_plan_rates_amounts_sorted: 0


In [52]:
merged_df = pd.merge(
    df_utility_rates_amounts,
    df_utility_plan_rates_amounts,
    how='left',
    on=['utility_id', 'created_at'],
    suffixes=('_utility', '_plan')
)

In [53]:
merged_df

,utility_id,utility_name_utility,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_value,utility_name_plan,supplier_id,supplier_name,...,term_end_date,plan_end_date,sq_ft_2600,sq_ft_800,plan_type,plan_rate_min_value,plan_rate_max_value,plan_rate_from_value,plan_rate_to_value,plan_rate_amount
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,Allegheny Power,1.0,FirstEnergy,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,NaN,NaN,NaN,4.40
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,Allegheny Power,2.0,Green Mountain Energy,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,NaN,NaN,NaN,4.04
2,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,Allegheny Power,2.0,Green Mountain Energy,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,1200,NaN,NaN,5.06
3,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,Allegheny Power,2.0,Green Mountain Energy,...,NaN,NaN,NaN,NaN,Regular Residential Service,1201.0,NaN,NaN,NaN,4.23
4,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,Allegheny Power,2.0,Green Mountain Energy,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,1200,NaN,NaN,5.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48105,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,West Penn Power,96.0,Town Square Energy,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,5.28
48106,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,West Penn Power,96.0,Town Square Energy,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.76
48107,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,West Penn Power,96.0,Town Square Energy,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.87
48108,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
merged_df.columns

Index(['utility_id', 'utility_name_utility', 'utility_rate_type', 'created_at',
       'utility_rate_from_value', 'utility_rate_to_value',
       'utility_rate_value', 'utility_name_plan', 'supplier_id',
       'supplier_name', 'plan_id', 'state', 'rate_unit', 'contact_number',
       'is_variable', 'rate_type_plan', 'is_green', 'description', 'term',
       'website_url', 'green_details', 'green_percentage', 'start_date',
       'service_type', 'term_end_date', 'plan_end_date', 'sq_ft_2600',
       'sq_ft_800', 'plan_type', 'plan_rate_min_value', 'plan_rate_max_value',
       'plan_rate_from_value', 'plan_rate_to_value', 'plan_rate_amount'],
      dtype='object')

In [55]:
required_columns = [
    'utility_id',
    'utility_name_utility',
    'utility_rate_type',
    'created_at',
    'utility_rate_from_value',
    'utility_rate_to_value',
    'utility_rate_value',
    'supplier_id',
    'supplier_name',
    'plan_id',
    'state',
    'rate_unit',
    'contact_number',
    'is_variable',
    'rate_type_plan',
    'is_green',
    'description',
    'term',
    'website_url',
    'green_details',
    'green_percentage',
    'start_date',
    'service_type',
    'term_end_date',
    'plan_end_date',
    'sq_ft_2600',
    'sq_ft_800',
    'plan_type',
    'plan_rate_min_value',
    'plan_rate_max_value',
    'plan_rate_from_value',
    'plan_rate_to_value',
    'plan_rate_amount']

In [56]:
merged_df = merged_df[required_columns].copy()

In [57]:
merged_df

,utility_id,utility_name_utility,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_value,supplier_id,supplier_name,plan_id,...,term_end_date,plan_end_date,sq_ft_2600,sq_ft_800,plan_type,plan_rate_min_value,plan_rate_max_value,plan_rate_from_value,plan_rate_to_value,plan_rate_amount
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,1.0,FirstEnergy,1.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,NaN,NaN,NaN,4.40
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,2.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,NaN,NaN,NaN,4.04
2,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,1200,NaN,NaN,5.06
3,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,1201.0,NaN,NaN,NaN,4.23
4,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,4.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,1200,NaN,NaN,5.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48105,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242820.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,5.28
48106,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242821.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.76
48107,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242822.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.87
48108,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
merged_df.rename(columns={
    'utility_name_utility': 'utility_name',
    'utility_rate_value': 'utility_rate_amount',
}, inplace=True)

In [59]:
merged_df

,utility_id,utility_name,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_amount,supplier_id,supplier_name,plan_id,...,term_end_date,plan_end_date,sq_ft_2600,sq_ft_800,plan_type,plan_rate_min_value,plan_rate_max_value,plan_rate_from_value,plan_rate_to_value,plan_rate_amount
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,1.0,FirstEnergy,1.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,NaN,NaN,NaN,4.40
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,2.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,NaN,NaN,NaN,4.04
2,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,1200,NaN,NaN,5.06
3,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,1201.0,NaN,NaN,NaN,4.23
4,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,4.0,...,NaN,NaN,NaN,NaN,Regular Residential Service,0.0,1200,NaN,NaN,5.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48105,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242820.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,5.28
48106,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242821.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.76
48107,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242822.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.87
48108,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
dataset_with_benefits = pd.merge(merged_df, df_benefits, on="plan_id", how="left")

In [61]:
drop_columns = [
    "benefit_id",
    ]
dataset_with_benefits.drop(columns=drop_columns, inplace=True)

In [62]:
dataset_with_benefits.rename(columns={
    'value': 'benefits_value',
}, inplace=True)

In [63]:
dataset_with_benefits

,utility_id,utility_name,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_amount,supplier_id,supplier_name,plan_id,...,sq_ft_2600,sq_ft_800,plan_type,plan_rate_min_value,plan_rate_max_value,plan_rate_from_value,plan_rate_to_value,plan_rate_amount,benefit_type,benefits_value
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,1.0,FirstEnergy,1.0,...,NaN,NaN,Regular Residential Service,0.0,NaN,NaN,NaN,4.40,NaN,NaN
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,2.0,...,NaN,NaN,Regular Residential Service,0.0,NaN,NaN,NaN,4.04,NaN,NaN
2,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,Regular Residential Service,0.0,1200,NaN,NaN,5.06,NaN,NaN
3,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,Regular Residential Service,1201.0,NaN,NaN,NaN,4.23,NaN,NaN
4,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,4.0,...,NaN,NaN,Regular Residential Service,0.0,1200,NaN,NaN,5.70,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48392,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242820.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,5.28,NaN,NaN
48393,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242821.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.76,NaN,NaN
48394,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242822.0,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,6.87,NaN,NaN
48395,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
df_amounts

,amount_id,from_value,to_value,value,rate_id,usage_credit_id,monthly_charge_id
0,1,NaN,NaN,3.221,1.0,NaN,NaN
1,2,NaN,NaN,4.400,2.0,NaN,NaN
2,3,NaN,NaN,0.000,NaN,1.0,NaN
3,4,NaN,NaN,4.040,3.0,NaN,NaN
4,5,NaN,NaN,0.000,NaN,2.0,NaN
...,...,...,...,...,...,...,...
1773977,1773978,NaN,NaN,6.369,659498.0,NaN,NaN
1773978,1773979,NaN,NaN,6.405,659499.0,NaN,NaN
1773979,1773980,NaN,NaN,6.369,659500.0,NaN,NaN
1773980,1773981,NaN,NaN,8.917,659501.0,NaN,NaN


In [65]:
df_monthly_charges_amounts = pd.merge(df_monthly_charges, df_amounts, on="monthly_charge_id", how="left")
drop_columns = [
    "monthly_charge_id",
    "amount_id",
    "rate_id",
    "usage_credit_id"
]


In [66]:
df_monthly_charges_amounts.columns

Index(['monthly_charge_id', 'min_value', 'max_value', 'plan_id', 'amount_id',
       'from_value', 'to_value', 'value', 'rate_id', 'usage_credit_id'],
      dtype='object')

In [67]:
df_monthly_charges_amounts.drop(columns=drop_columns, inplace=True)

In [68]:
df_monthly_charges_amounts.rename(columns={
    "value": "monthly_charge_amount",
    "min_value": "monthly_charge_min_value",
    "max_value": "monthly_charge_max_value",
    "from_value": "monthly_charge_from_value",
    "to_value": "monthly_charge_to_value"
}, inplace=True)

In [69]:
df_monthly_charges_amounts.columns

Index(['monthly_charge_min_value', 'monthly_charge_max_value', 'plan_id',
       'monthly_charge_from_value', 'monthly_charge_to_value',
       'monthly_charge_amount'],
      dtype='object')

In [70]:
dataset_with_monthly_charges = pd.merge(dataset_with_benefits, df_monthly_charges_amounts, on="plan_id", how="left")

In [71]:
dataset_with_monthly_charges

,utility_id,utility_name,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_amount,supplier_id,supplier_name,plan_id,...,plan_rate_from_value,plan_rate_to_value,plan_rate_amount,benefit_type,benefits_value,monthly_charge_min_value,monthly_charge_max_value,monthly_charge_from_value,monthly_charge_to_value,monthly_charge_amount
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,1.0,FirstEnergy,1.0,...,NaN,NaN,4.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,2.0,...,NaN,NaN,4.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,5.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,4.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,4.0,...,NaN,NaN,5.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48392,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242820.0,...,NaN,NaN,5.28,NaN,NaN,0.0,NaN,NaN,NaN,0.0
48393,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242821.0,...,NaN,NaN,6.76,NaN,NaN,0.0,NaN,NaN,NaN,0.0
48394,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242822.0,...,NaN,NaN,6.87,NaN,NaN,0.0,NaN,NaN,NaN,0.0
48395,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
drop_columns = [
    "usage_credit_id",
]
df_usage_credits.drop(columns=drop_columns, inplace=True)


In [73]:
df_usage_credits.rename(columns=({
    "min_value": "usage_credit_min_value",
    "max_value": "usage_credit_max_value",
}), inplace=True)

In [74]:
dataset_with_usage_credits = pd.merge(dataset_with_monthly_charges, df_usage_credits, on="plan_id", how="left")

In [75]:
dataset_with_usage_credits

,utility_id,utility_name,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_amount,supplier_id,supplier_name,plan_id,...,plan_rate_amount,benefit_type,benefits_value,monthly_charge_min_value,monthly_charge_max_value,monthly_charge_from_value,monthly_charge_to_value,monthly_charge_amount,usage_credit_min_value,usage_credit_max_value
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,1.0,FirstEnergy,1.0,...,4.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,2.0,...,4.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
2,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,5.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
3,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,4.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
4,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,4.0,...,5.70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48392,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242820.0,...,5.28,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN
48393,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242821.0,...,6.76,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN
48394,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242822.0,...,6.87,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,NaN
48395,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [76]:
df_early_term_fees

,early_term_fee_id,min_value,max_value,fee_type,term,plan_id
0,1,0.0,NaN,NaN,NaN,5123
1,2,0.0,NaN,NaN,NaN,5125
2,3,0.0,NaN,NaN,NaN,5126
3,4,0.0,NaN,NaN,NaN,5127
4,5,0.0,NaN,NaN,NaN,5131
...,...,...,...,...,...,...
533229,533230,0.0,NaN,straight,NaN,656737
533230,533231,0.0,NaN,straight,NaN,656738
533231,533232,0.0,NaN,straight,NaN,656739
533232,533233,12500.0,NaN,straight,NaN,656740


In [77]:
drop_columns = [
    "early_term_fee_id",
]
df_early_term_fees.drop(columns=drop_columns, inplace=True)

df_early_term_fees.rename(columns=({
    "min_value": "early_term_fees_min_value",
    "max_value": "early_term_fees_max_value",
    "fee_type": "early_term_fee_type",
    "term": "early_term_fee_term"
}), inplace=True)

In [78]:
dataset_with_early_term_fees = pd.merge(dataset_with_usage_credits, df_early_term_fees, on="plan_id", how="left")

In [79]:
dataset_with_early_term_fees

,utility_id,utility_name,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_amount,supplier_id,supplier_name,plan_id,...,monthly_charge_max_value,monthly_charge_from_value,monthly_charge_to_value,monthly_charge_amount,usage_credit_min_value,usage_credit_max_value,early_term_fees_min_value,early_term_fees_max_value,early_term_fee_type,early_term_fee_term
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,1.0,FirstEnergy,1.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,2.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,3.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,2.0,Green Mountain Energy,4.0,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48392,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242820.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
48393,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242821.0,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
48394,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,96.0,Town Square Energy,242822.0,...,NaN,NaN,NaN,0.0,0.0,NaN,5000.0,NaN,straight,NaN
48395,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
dataset_with_early_term_fees.drop(columns=["plan_id","supplier_id"], inplace=True)

In [81]:
dataset_with_early_term_fees

,utility_id,utility_name,utility_rate_type,created_at,utility_rate_from_value,utility_rate_to_value,utility_rate_amount,supplier_name,state,rate_unit,...,monthly_charge_max_value,monthly_charge_from_value,monthly_charge_to_value,monthly_charge_amount,usage_credit_min_value,usage_credit_max_value,early_term_fees_min_value,early_term_fees_max_value,early_term_fee_type,early_term_fee_term
0,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,FirstEnergy,pa,kWh,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
1,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,Green Mountain Energy,pa,kWh,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,Green Mountain Energy,pa,kWh,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,Green Mountain Energy,pa,kWh,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,1,Allegheny Power,default rate,1999-07-30,NaN,NaN,3.221,Green Mountain Energy,pa,kWh,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48392,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,Town Square Energy,pa,kWh,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
48393,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,Town Square Energy,pa,kWh,...,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN
48394,10,West Penn Power,default rate,2020-04-06,NaN,NaN,6.059,Town Square Energy,pa,kWh,...,NaN,NaN,NaN,0.0,0.0,NaN,5000.0,NaN,straight,NaN
48395,10,West Penn Power,default rate,2023-09-14,NaN,NaN,9.929,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [82]:
try:
  pickle_file_path = 'WattBuyData1999_2024.pkl'
  dataset_with_early_term_fees.to_pickle(pickle_file_path)
  print(f"DataFrame successfully saved as '{pickle_file_path}'.")
except Exception as e:
    print(f"An error occurred: {e}")

DataFrame successfully saved as 'WattBuyData1999_2024.pkl'.
